In [1]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_dir = './FMCG/train/images'
csv_data = pd.read_csv('./FMCG/train/_classes.csv')
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [3]:
csv_data.columns

Index(['filename', ' AfterShave', ' FabricBrightener', ' FaceMask',
       ' Handwash', ' Lotion', ' Napkin', ' Oil', ' PetroliumJelly', ' Powder',
       ' Soap', ' Sweetner', ' Toothpaste'],
      dtype='object')

In [4]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=csv_data,
    directory=train_dir,
    x_col='filename',
    y_col=[' AfterShave', ' FabricBrightener', ' FaceMask', ' Handwash', ' Lotion', ' Napkin', ' Oil', ' PetroliumJelly', ' Powder', ' Soap', ' Sweetner', ' Toothpaste'],  # Keep the spaces
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    shuffle=True
)

Found 202 validated image filenames.


In [5]:
test_dir = './FMCG/test/images'  # Path to test images
test_csv = './FMCG/test/_classes.csv'  # CSV containing test filenames and labels

test_data = pd.read_csv(test_csv)

In [6]:
test_generator = datagen.flow_from_dataframe(
    dataframe=test_data,
    directory=test_dir,
    y_col=[' AfterShave', ' FabricBrightener', ' FaceMask', ' Handwash', ' Lotion', ' Napkin', ' Oil', ' PetroliumJelly', ' Powder', ' Soap', ' Sweetner', ' Toothpaste'],  # Keep the spaces
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',  
    shuffle=False 
)

Found 24 validated image filenames.


In [51]:
import tensorflow as tf

# Load a pre-trained model and fine-tune it
base_model = tf.keras.applications.MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

# Define the new model
model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.25),  # Increase dropout rate
    tf.keras.layers.Dense(12, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with validation data
history = model.fit(
    train_generator,
    epochs=25
)


Epoch 1/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 61ms/step - accuracy: 0.1187 - loss: 0.5351
Epoch 2/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.2315 - loss: 0.2978
Epoch 3/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.3181 - loss: 0.2729
Epoch 4/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.5288 - loss: 0.2248
Epoch 5/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.4892 - loss: 0.2071
Epoch 6/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.6477 - loss: 0.1737
Epoch 7/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - accuracy: 0.7309 - loss: 0.1474
Epoch 8/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.7417 - loss: 0.1358
Epoch 9/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.8648 - loss: 0.1151
Epoch 10/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.9066 - loss: 0.1036
Epoch 11/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.9331 - loss: 0.0954
Epoch 12/25
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.9437 - loss: 0.0834
E

In [52]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.7083 - loss: 0.1712
Test Loss: 0.17116080224514008, Test Accuracy: 0.7083333134651184


In [53]:
predictions = model.predict(test_generator)
predicted_classes = predictions.argmax(axis=-1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [54]:
class_labels = [' AfterShave', ' FabricBrightener', ' FaceMask', ' Handwash', ' Lotion', ' Napkin', ' Oil', ' PetroliumJelly', ' Powder', ' Soap', ' Sweetner', ' Toothpaste']  # Keep the spaces
    

predicted_class_names = [class_labels[predicted_index] for predicted_index in predicted_classes]

for filename, predicted_class in zip(test_data['filename'], predicted_class_names):
    print(f'Filename: {filename}, Predicted Class: {predicted_class}')

Filename: images-1-_jpeg.rf.27fd9e5aa4f310481f20e23895d9d15c.jpg, Predicted Class:  PetroliumJelly
Filename: download-5-_jpeg.rf.9b69ce4dbbb2058f38d42ec41caeb398.jpg, Predicted Class:  Napkin
Filename: images_jpeg.rf.6a81f460b00de4513ede32c01948f2fb.jpg, Predicted Class:  Powder
Filename: hq720_jpg.rf.452a8276c4b12dd07d2c472c2b332ddd.jpg, Predicted Class:  PetroliumJelly
Filename: download_jpeg.rf.b3edaf1cfb9702849cf424e9b5b3ee44.jpg, Predicted Class:  AfterShave
Filename: download-9-_jpeg.rf.9de97aa023aa35128ba21b33160d83ae.jpg, Predicted Class:  Napkin
Filename: 8941100501214_jpg.rf.9e879b8637215a9e7b9a8fcd4ba4a70a.jpg, Predicted Class:  Toothpaste
Filename: chamak-top-img-new-update-2023_png.rf.3519ad81a56c7b9c87895fd3bd19c9f6.jpg, Predicted Class:  Lotion
Filename: OOJ44hHIeX6fowKcTrMoQj56fXi7xkZiRl7FvqkA_jpg.rf.c7ad5a7ce486556eefd9563745559c69.jpg, Predicted Class:  Powder
Filename: 8def6e4622fca704e114c6785f3013dd_jpg.rf.1de911766e40fe266d60b5df7ae23c13.jpg, Predicted Class:  Han

In [27]:
model.save('FMCG.h5')